In [1745]:
import numpy as np

In [1746]:
def forward_layer_matrixMult(x, w):
    # both w,x are row vectors
    # x : n * d, w : d * 1
    # N = xw

    return x @ w

In [1747]:
def backward_layer_matrixMult(x, w):
    # N = xw
    # dN/dw, dN/dx

    return x, w

In [1748]:
def forward_layer_biasAdd(N, b):
    # N : n * 1, b : n * 1
    # P = N + b

    return N + b

In [1749]:
def backward_layer_biasAdd(N, b):
    # P = N + b (n * 1)
    # dP/dN = Identity (n * n)
    # dP/db = 1 (n * 1)

    return np.identity(N.shape[0]), np.ones((N.shape[0], 1))

In [1750]:
def forward_layer_sigmoid(P):
    # Q = 1 / 1 + e^-P_i
    
    return 1 / (1 + np.exp(-P))

In [1751]:
def backward_layer_sigmoid(P):
    # dQ_i/dP_i = Q_i * (1 - Q_i)

    Q = forward_layer_sigmoid(P)
    Q1 = Q * (1 - Q)
    
    return np.diag(Q1)

In [1752]:
def forward_layer_softMax(Q):
    denom = np.sum(np.exp(Q))
    return np.exp(Q) / denom

In [1753]:
def backward_layer_softMax():
    pass

In [1754]:
def forward_layer_meanSqrLoss(P,y):
    # P : n * 1, y : n * 1
    # L = 1/n * sum((P - y) ^ 2)    : (1 * 1)
    
    return np.sum((P - y)**2)

In [1755]:
def backward_layer_meanSqrLoss(P, y):
    # L = 1/n * sum((P - y) ^ 2)    : (1 * 1)
    # dL/dP = 2 * (P - y) : (n * 1)

    return 2 * (P - y)

In [1756]:
def forward_layer_crossEntropy(P,y):
    #y can be multiclass or singleclass
    #P,y both are row vectors
    #    or
    #P,y both are matrix of same shape
        # returns loss_vector
    if(len(y.shape) == 1):
        loss = np.log((P ** y) * ((1 - P) ** (1 - y)))
        return np.sum(loss)
    else:
        loss = np.log((P ** y))
        return np.sum(loss) 

In [1757]:
#def backward_layer_crossEntropy()

### Q2 Boston Dataset

In [1758]:
import pandas as pd
import numpy as np
import sklearn.datasets as sk

In [1759]:
dictt = sk.fetch_california_housing()
data = dictt.data
target = dictt.target
data.shape

(20640, 8)

In [1773]:
from sklearn.preprocessing import Normalizer

In [1774]:
#Normalising dataset

transformer = Normalizer().fit(data)

# mean_vals = np.mean(data, axis=0)
# std_devs = np.std(data, axis=0)
# epsilon = 1e-8
# data = (data - mean_vals) / (std_devs + epsilon)

In [1777]:
data = transformer.transform(data)

In [1778]:
valid = 20000
test_x = data[valid+1:]
test_y = target[valid+1:]
x = data[0:valid]
y = target[0:valid]

In [1779]:
def forwardProp(x,w,b,y):
    N = forward_layer_matrixMult(x,w)
    P = forward_layer_biasAdd(N,b)
    L = forward_layer_meanSqrLoss(P,y)
    print("Error:", L/y.shape[0])
    return N,P

In [1780]:
def backwardProp(x,w,b,N,P,y):
    dL_dP = backward_layer_meanSqrLoss(P,y)
    dP_dN,dP_dB = backward_layer_biasAdd(N,b)
    dN_dw,dN_dx = backward_layer_matrixMult(x,w)
    dl_dw = dN_dw.T @ dP_dN @ dL_dP
    dl_db =  dP_dB.T @ dL_dP
    return dl_dw,dl_db

In [1781]:
def stochastic_gd(x,w,b,y,alpha,epoch):
    for j in range(epoch):
        i = np.random.randint(y.shape[0])
        x_ar = x[i].reshape(1,-1)
        y_ar = np.array([y[i]])
        N,P = forwardProp(x_ar,w,b,y_ar)
        # del_w=2*(y[i]-P) @ x_ar
        # del_b=2*(y[i]-P)
        del_w,del_b = backwardProp(x_ar,w,b,N,P,y_ar)
        # print("del w, del b : ", del_w, del_b)
        w = w - (del_w * alpha)
        b = b - (del_b * alpha)
    return w, b

In [1782]:
w = np.random.rand(len(data[0]),1)
b = np.random.rand(1,1)
alpha = 1e-5
epoch = 30000
w_ = w

In [1783]:
w,b = stochastic_gd(x,w,b,y,alpha,epoch)

Error: 0.6711032519118671
Error: 1.6917442557071347
Error: 0.15148245137132438
Error: 11.537577885740825
Error: 0.3546057104670182
Error: 1.1214694778243326
Error: 12.652699515997416
Error: 0.11155301396779034
Error: 13.47159393646971
Error: 1.1580884807315606
Error: 25.482075969035797
Error: 3.9904055578856443
Error: 1.9332173595017235
Error: 4.896701164569169
Error: 2.384586058978095
Error: 0.5704470926642553
Error: 4.237782643276217
Error: 1.3795166767975113
Error: 21.26281640840009
Error: 22.81124063524364
Error: 4.170365577195121
Error: 9.86451862539379
Error: 0.37941565490021895
Error: 3.896480934293938
Error: 0.3485082641838797
Error: 10.943132746999598
Error: 0.06248653411195551
Error: 3.423723536726827
Error: 0.00015809664926850158
Error: 1.098768110248867
Error: 0.04291352938457794
Error: 24.528491978574728
Error: 0.2896989594449196
Error: 1.554553571218478
Error: 12.399019172631725
Error: 17.266908810567177
Error: 3.1132807555064335
Error: 3.769129075963726
Error: 21.4701851

In [1784]:
w

array([[0.23428053],
       [0.633619  ],
       [0.64108708],
       [0.92368637],
       [0.59526935],
       [0.65677883],
       [0.03002308],
       [0.4705288 ]])

In [1785]:
N,P = forwardProp(test_x,w,b,test_y)

Error: 1507.8330222807654


### Q3 IRIS

In [ ]:
d = sk.load_iris()
x = d.data
y = d.target

In [ ]:
x

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [ ]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [ ]:
def oneHot_encoding(categories ,labels):
    num_samples = len(labels)
    num_categories = len(categories)
    one_hot = np.zeros((num_samples, num_categories))

    # Perform one-hot encoding
    for i, label in enumerate(labels):
        index = categories.index(label)
        one_hot[i, index] = 1
    return one_hot

In [ ]:
y_encoded = oneHot_encoding([0,1,2],y)
y_encoded

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0